# YMBOT-133 - experiment on IMX219 sensor quality

## Prepare:
1. Imports
2. Aliases
3. Data paths and results

In [1]:
import os
from datetime import datetime
import pandas as pd

from ymbot_cv_utils.toloka.outputs import PolygonOutput
from ymbot_cv_utils.toloka.image_cropper import TolokaImageCropper
from ymbot_cv_utils.ya_pic_decoder.barcode_decoder import BarcodeDecoder
from ymbot_cv_utils.utils.image_loader import ImageLoader
from ymbot_cv_utils.ya_pic_decoder.code_info import CodeInfo

In [2]:
# useful aliases for Toloka lebeling file

a_input = "INPUT:image"
a_output = "OUTPUT:result"
a_status = "ASSIGNMENT:status"
a_started_dt = "ASSIGNMENT:started"
a_submited_dt = "ASSIGNMENT:submitted"
a_accepted_dt = "ASSIGNMENT:accepted"
a_skipped_dt = "ASSIGNMENT:skipped"

a_approved_status = "APPROVED"
a_skipped_status = "SKIPPED"

# labeling types
a_plt_qr = "plt_qr"
a_plt_bar = "plt_bar"
a_drp = "drp"

# result columns
a_res_type = "type"
a_res_dist = "distance"
a_res_cond = "condition"
a_res_expected = "expected"
a_res_result = "result"
a_res_code_type = "code_type"
a_res_coincidence = "coincidence"
a_res_time = "label_time(sec)"
a_res_filename = "crop_filename"


In [3]:
# prepare files from https://wiki.yandex-team.ru/roboticsmarket/cv/research/experiments/experimentimx290/#rezultatyrazmetki
tsv_files = {
    a_plt_qr: "data/assignments_from_pool_1118587__02-03-2022_PLT_QR.tsv",
    a_plt_bar: "data/assignments_from_pool_1118626__02-03-2022 PLT_BAR.tsv",
    a_drp: "data/assignments_from_pool_1118655__02-03-2022_DRP.tsv"
}

expected_results = {
    a_plt_qr: "PLT0065762",
    a_plt_bar: "PLT0065761",
    a_drp: "DRP0002571",
}


## Experiments structures
1. LightConditionType - discribes light condition as Enum
2. FrameInfo - filename parser to components:
   - distance
   - timestamp
   - condition
   - frame_id

In [4]:
from enum import Enum


class LightConditionType(Enum):
    DARK = "dark"
    LIGHT = "light"
    DIRECT = "direct"


class FrameInfo:
    _distance: int
    _frame_id: int
    _ts: float
    _condition: LightConditionType

    @property
    def distance(self):
        return self._distance

    @property
    def ts(self):
        return self._ts

    @property
    def condition(self):
        return self._condition.value

    @property
    def frame_id(self):
        return self._frame_id

    def __init__(self, filename: str) -> None:
        filename = filename.split('/')[-1]
        parts = filename.split('_')
        self._distance = int(parts[0].replace('cm', ''))
        self._frame_id = int(parts[1])
        self._ts = float(parts[2])
        self._condition = LightConditionType(parts[3].split('.')[0])

    def __repr__(self):
        return f"Frame {self._frame_id} on range {self._distance} with condition {self._condition.value}"

# fast check:
frame_info = FrameInfo(
    "https://ie.wampi.ru/2022/03/02/125cm_5001_1644930303.7325857_dark.jpg")
frame_info, frame_info.condition, frame_info.frame_id, frame_info.ts, frame_info.distance

(Frame 5001 on range 125 with condition dark,
 'dark',
 5001,
 1644930303.7325857,
 125)

## Calculating

1. Create empty dataframe
2. Iterate on each type of labeled codes
   1. Open and prepare df with tsv data
   2. For each record:
      - loading image
      - crop
      - decoding
      - write results


In [5]:
BASE_DIR = "./crops"
os.makedirs(BASE_DIR, exist_ok=True)

columns = [a_res_type,
           a_res_dist,
           a_res_cond,
           a_res_expected,
           a_res_result,
           a_res_code_type,
           a_res_coincidence,
           a_res_time,
           a_res_filename,
           a_input]
res_df = pd.DataFrame(columns=columns)

label_types = [a_plt_bar, a_plt_qr, a_drp]

for label_type in label_types:

    # tsv df prepare
    df = pd.read_csv(tsv_files[label_type], sep='\t')
    df = df.dropna(how='all', axis=0).dropna(how='all', axis=1)
    df.reset_index(drop=True)

    for index, row in df.iterrows():
        if row[a_status] == a_approved_status:
            image = ImageLoader.load(row[a_input])
            crop = TolokaImageCropper.get_crop(image, row[a_output], PolygonOutput)
            codes = BarcodeDecoder.decode(crop)
            code = CodeInfo() if len(codes) != 1 else codes[0]
            coincidence = code.value == expected_results[label_type]
            frame_info = FrameInfo(row[a_input])
            crop_filename = os.path.join(
                BASE_DIR, f"{label_type}_{frame_info.distance}_{frame_info.frame_id}_{frame_info.condition}.jpg")
            note = {
                a_res_type: label_type,
                a_res_dist: frame_info.distance,
                a_res_cond: frame_info.condition,
                a_res_expected: expected_results[label_type],
                a_res_result: "-" if not code else code.value,
                a_res_code_type: "-" if not code else code.type,
                a_res_coincidence: 1 if coincidence else 0,
                a_res_time: (datetime.fromisoformat(row[a_accepted_dt]) - datetime.fromisoformat(row[a_started_dt])).total_seconds(),
                a_res_filename: crop_filename,
                a_input: row[a_input]
            }
            res_df.loc[len(res_df)] = note
            crop.save(crop_filename)

        elif row[a_status] == a_skipped_status:
            pass


In [6]:
# record results to YMBOT-133 folder
res_df.sort_values(by=[a_res_type,
                       a_res_coincidence,
                       a_res_dist,
                       a_res_cond, ]).to_excel("output.xlsx")
